# Quality Analysis System - By Satellite Type

This notebook analyzes data quality separately by satellite type:
- **S1_GRD**: area_km2 calculated by radar method
- **LANDSAT**: area_km2 and ndwi_area_km2 calculated by optical methods

Each location is analyzed independently for outliers.

In [1]:
# Import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import sqlite3

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Libraries imported successfully!")

Libraries imported successfully!


In [2]:
# Load data and separate by satellite type
db_path = r"sqlite:///db_1.db"
engine = create_engine(db_path, echo=False)

df = pd.read_sql('SELECT * FROM water_surface_detection_v3', con=engine)

print(f"Total records: {len(df)}")
print(f"Unique locations: {df['loc'].nunique()}")
print(f"\nColumns: {df.columns.tolist()}")

# IMPORTANT: Separate by satellite type (different calculation methods)
df_s1 = df[df['sat_id'] == 'S1_GRD'].copy()
df_landsat = df[df['sat_id'] != 'S1_GRD'].copy()

print(f"\n{'='*80}")
print(f"S1_GRD records: {len(df_s1)} ({len(df_s1)/len(df)*100:.1f}%)")
print(f"  - Unique locations: {df_s1['loc'].nunique()}")
print(f"  - Method: Radar-based area calculation")

print(f"\nLandsat records: {len(df_landsat)} ({len(df_landsat)/len(df)*100:.1f}%)")
print(f"  - Unique locations: {df_landsat['loc'].nunique()}")
print(f"  - Satellite types: {df_landsat['sat_id'].unique()}")
print(f"  - Methods: Optical-based area_km2 and ndwi_area_km2")

# Check ndwi_area_km2 availability
if 'ndwi_area_km2' in df.columns:
    print(f"\nndwi_area_km2 column found:")
    print(f"  - Non-null in S1_GRD: {df_s1['ndwi_area_km2'].notna().sum()}")
    print(f"  - Non-null in Landsat: {df_landsat['ndwi_area_km2'].notna().sum()}")
else:
    print(f"\nWarning: ndwi_area_km2 column not found")

Total records: 160055
Unique locations: 76

Columns: ['time', 'loc', 'image_id', 'sat_id', 'resolution', 'n_pixels', 'n_nan', 'n_nan_lim', 'sd_nan', 'error', 'area_km2', 'error_vis', 'ndwi_area_km2']

S1_GRD records: 95225 (59.5%)
  - Unique locations: 76
  - Method: Radar-based area calculation

Landsat records: 64830 (40.5%)
  - Unique locations: 76
  - Satellite types: ['LC08' 'LC09' 'LE07' 'LT05']
  - Methods: Optical-based area_km2 and ndwi_area_km2

ndwi_area_km2 column found:
  - Non-null in S1_GRD: 0
  - Non-null in Landsat: 47202


In [3]:
# Data quality assessment by satellite type
for sat_type, df_subset in [("S1_GRD", df_s1), ("LANDSAT", df_landsat)]:
    print(f"\n{'='*80}")
    print(f"DATA QUALITY - {sat_type}")
    print(f"{'='*80}")
    
    print(f"\narea_km2 statistics:")
    print(df_subset['area_km2'].describe())
    print(f"  Negative values: {(df_subset['area_km2'] < 0).sum()}")
    print(f"  Zero values: {(df_subset['area_km2'] == 0).sum()} ({(df_subset['area_km2'] == 0).sum()/len(df_subset)*100:.2f}%)")
    print(f"  Missing values: {df_subset['area_km2'].isnull().sum()}")
    
    if sat_type == "LANDSAT" and 'ndwi_area_km2' in df_subset.columns:
        print(f"\nndwi_area_km2 statistics:")
        print(df_subset['ndwi_area_km2'].describe())
        print(f"  Negative values: {(df_subset['ndwi_area_km2'] < 0).sum()}")
        print(f"  Zero values: {(df_subset['ndwi_area_km2'] == 0).sum()} ({(df_subset['ndwi_area_km2'] == 0).sum()/df_subset['ndwi_area_km2'].notna().sum()*100:.2f}%)")
        print(f"  Missing values: {df_subset['ndwi_area_km2'].isnull().sum()} ({df_subset['ndwi_area_km2'].isnull().sum()/len(df_subset)*100:.2f}%)")
    
    print(f"\nRecords per location:")
    print(df_subset.groupby('loc').size().describe())


DATA QUALITY - S1_GRD

area_km2 statistics:
count    95225.000000
mean         7.964681
std         23.303288
min          0.000000
25%          0.208500
50%          0.927200
75%          5.860800
max        393.328000
Name: area_km2, dtype: float64
  Negative values: 0
  Zero values: 14 (0.01%)
  Missing values: 0

Records per location:
count      76.000000
mean     1252.960526
std       328.343700
min       616.000000
25%       966.250000
50%      1296.000000
75%      1436.000000
max      1923.000000
dtype: float64

DATA QUALITY - LANDSAT

area_km2 statistics:
count    64830.000000
mean         6.062493
std         20.881164
min          0.000000
25%          0.040500
50%          0.309600
75%          2.170800
max        415.639800
Name: area_km2, dtype: float64
  Negative values: 0
  Zero values: 7784 (12.01%)
  Missing values: 0

ndwi_area_km2 statistics:
count    47202.000000
mean         6.221253
std         21.500183
min          0.000000
25%          0.043200
50%          0.

In [4]:
# Outlier detection functions
def detect_outliers_iqr(data, column, multiplier=1.5):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    data[f'{column}_outlier_iqr'] = ((data[column] < lower_bound) | (data[column] > upper_bound)).astype(int)
    data[f'{column}_lower_bound'] = lower_bound
    data[f'{column}_upper_bound'] = upper_bound
    return data

def detect_outliers_zscore(data, column, threshold=3):
    mean = data[column].mean()
    std = data[column].std()
    data[f'{column}_zscore'] = np.abs((data[column] - mean) / std)
    data[f'{column}_outlier_zscore'] = (data[f'{column}_zscore'] > threshold).astype(int)
    return data

def detect_outliers_mad(data, column, threshold=3.5):
    median = data[column].median()
    mad = np.median(np.abs(data[column] - median))
    if mad == 0:
        data[f'{column}_mad_score'] = 0
        data[f'{column}_outlier_mad'] = 0
    else:
        modified_zscore = 0.6745 * (data[column] - median) / mad
        data[f'{column}_mad_score'] = np.abs(modified_zscore)
        data[f'{column}_outlier_mad'] = (data[f'{column}_mad_score'] > threshold).astype(int)
    return data

def detect_outliers_by_location(df, column, methods=['iqr', 'zscore', 'mad']):
    results = []
    for loc in df['loc'].unique():
        loc_data = df[df['loc'] == loc].copy()
        if len(loc_data) < 5:
            print(f"Warning: Location {loc} has only {len(loc_data)} records. Skipping outlier detection.")
            loc_data[f'{column}_outlier_combined'] = 0
            results.append(loc_data)
            continue
        
        if 'iqr' in methods:
            loc_data = detect_outliers_iqr(loc_data, column)
        if 'zscore' in methods:
            loc_data = detect_outliers_zscore(loc_data, column)
        if 'mad' in methods:
            loc_data = detect_outliers_mad(loc_data, column)
        
        outlier_cols = [c for c in loc_data.columns if c.startswith(f'{column}_outlier_') and not c.endswith('_combined')]
        if len(outlier_cols) > 0:
            loc_data[f'{column}_outlier_combined'] = (loc_data[outlier_cols].sum(axis=1) >= 2).astype(int)
        
        results.append(loc_data)
    return pd.concat(results, ignore_index=True)

print("Outlier detection functions defined!")

Outlier detection functions defined!


In [5]:
# Apply outlier detection - S1_GRD
print("="*80)
print("OUTLIER DETECTION - S1_GRD (area_km2)")
print("="*80)

df_s1_analyzed = detect_outliers_by_location(df_s1, 'area_km2', methods=['iqr', 'zscore', 'mad'])

print(f"\nTotal outliers: {df_s1_analyzed['area_km2_outlier_combined'].sum()} ({df_s1_analyzed['area_km2_outlier_combined'].sum()/len(df_s1_analyzed)*100:.2f}%)")
outliers_by_loc = df_s1_analyzed[df_s1_analyzed['area_km2_outlier_combined'] == 1].groupby('loc').size().sort_values(ascending=False)
print(f"Locations with outliers: {len(outliers_by_loc)}")
print(f"\nTop 10 locations:\n{outliers_by_loc.head(10)}")

OUTLIER DETECTION - S1_GRD (area_km2)

Total outliers: 2588 (2.72%)
Locations with outliers: 67

Top 10 locations:
loc
1676    248
1673    145
1257    133
456     132
706     125
704     120
641     116
1677    110
871      95
599      80
dtype: int64


In [6]:
# Apply outlier detection - Landsat area_km2
print("="*80)
print("OUTLIER DETECTION - LANDSAT (area_km2)")
print("="*80)

df_landsat_analyzed = detect_outliers_by_location(df_landsat, 'area_km2', methods=['iqr', 'zscore', 'mad'])

print(f"\nTotal outliers: {df_landsat_analyzed['area_km2_outlier_combined'].sum()} ({df_landsat_analyzed['area_km2_outlier_combined'].sum()/len(df_landsat_analyzed)*100:.2f}%)")
outliers_by_loc = df_landsat_analyzed[df_landsat_analyzed['area_km2_outlier_combined'] == 1].groupby('loc').size().sort_values(ascending=False)
print(f"Locations with outliers: {len(outliers_by_loc)}")
print(f"\nTop 10 locations:\n{outliers_by_loc.head(10)}")

OUTLIER DETECTION - LANDSAT (area_km2)

Total outliers: 2013 (3.11%)
Locations with outliers: 68

Top 10 locations:
loc
1256    147
1911    145
1912    126
2039    115
1262    100
1913     96
2038     85
448      69
1679     66
1676     60
dtype: int64


In [7]:
# Apply outlier detection - Landsat ndwi_area_km2
if 'ndwi_area_km2' in df_landsat.columns:
    print("="*80)
    print("OUTLIER DETECTION - LANDSAT (ndwi_area_km2)")
    print("="*80)
    
    df_landsat_analyzed = detect_outliers_by_location(df_landsat_analyzed, 'ndwi_area_km2', methods=['iqr', 'zscore', 'mad'])
    
    print(f"\nTotal outliers: {df_landsat_analyzed['ndwi_area_km2_outlier_combined'].sum()} ({df_landsat_analyzed['ndwi_area_km2_outlier_combined'].sum()/len(df_landsat_analyzed)*100:.2f}%)")
    outliers_by_loc = df_landsat_analyzed[df_landsat_analyzed['ndwi_area_km2_outlier_combined'] == 1].groupby('loc').size().sort_values(ascending=False)
    print(f"Locations with outliers: {len(outliers_by_loc)}")
    print(f"\nTop 10 locations:\n{outliers_by_loc.head(10)}")
else:
    print("ndwi_area_km2 column not found - skipping")

OUTLIER DETECTION - LANDSAT (ndwi_area_km2)

Total outliers: 416 (0.64%)
Locations with outliers: 66

Top 10 locations:
loc
448     38
2338    35
1256    20
704     14
234     12
1262    12
458     12
1680    11
1679    11
1912    10
dtype: int64


In [8]:
# Quality metrics function
def calculate_location_quality_metrics(df, column):
    quality_metrics = []
    for loc in df['loc'].unique():
        loc_data = df[df['loc'] == loc]
        metrics = {
            'loc': loc,
            'n_records': len(loc_data),
            'mean': loc_data[column].mean(),
            'median': loc_data[column].median(),
            'std': loc_data[column].std(),
            'min': loc_data[column].min(),
            'max': loc_data[column].max(),
            'range': loc_data[column].max() - loc_data[column].min(),
            'cv': loc_data[column].std() / loc_data[column].mean() if loc_data[column].mean() != 0 else np.nan,
            'n_outliers': loc_data[f'{column}_outlier_combined'].sum() if f'{column}_outlier_combined' in loc_data.columns else 0,
            'outlier_rate': loc_data[f'{column}_outlier_combined'].sum() / len(loc_data) * 100 if f'{column}_outlier_combined' in loc_data.columns else 0,
            'n_zeros': (loc_data[column] == 0).sum(),
            'zero_rate': (loc_data[column] == 0).sum() / len(loc_data) * 100
        }
        quality_metrics.append(metrics)
    return pd.DataFrame(quality_metrics).sort_values('outlier_rate', ascending=False)

# Generate quality reports
print("="*80)
print("QUALITY REPORTS BY LOCATION")
print("="*80)

quality_s1 = calculate_location_quality_metrics(df_s1_analyzed, 'area_km2')
quality_landsat_area = calculate_location_quality_metrics(df_landsat_analyzed, 'area_km2')

print("\nS1_GRD - Top 10 locations by outlier rate:")
print(quality_s1.head(10))

print("\nLANDSAT area_km2 - Top 10 locations by outlier rate:")
print(quality_landsat_area.head(10))

if 'ndwi_area_km2_outlier_combined' in df_landsat_analyzed.columns:
    quality_landsat_ndwi = calculate_location_quality_metrics(df_landsat_analyzed, 'ndwi_area_km2')
    print("\nLANDSAT ndwi_area_km2 - Top 10 locations by outlier rate:")
    print(quality_landsat_ndwi.head(10))

QUALITY REPORTS BY LOCATION

S1_GRD - Top 10 locations by outlier rate:
     loc  n_records        mean    median        std      min       max  \
13  1676       1425    0.767400    0.6509   0.303804   0.0736    3.4306   
72   706        880  131.194984  139.5228  20.066035  11.7241  142.0503   
44   456       1318   24.472674   25.1931   2.537337   4.2993   26.1332   
10  1673       1449    0.298289    0.0317   0.557671   0.0000    4.0270   
58   704       1288    0.285565    0.2981   0.046485   0.0208    0.3346   
54   641       1293    8.958822    9.3799   1.474384   0.7283   10.3670   
14  1677       1273    5.473374    5.8398   1.297726   0.1800    7.4697   
63  2453        667   61.134865   66.3805  13.990807   4.2964   73.9283   
52   599       1041    1.161414    0.8817   1.075433   0.0000    4.9401   
2   1257       1819    1.051689    1.0693   0.141966   0.0470    1.7191   

       range        cv  n_outliers  outlier_rate  n_zeros  zero_rate  
13    3.3570  0.395887         

In [9]:
# Export results
print("="*80)
print("EXPORTING RESULTS")
print("="*80)

# S1_GRD
outliers_s1 = df_s1_analyzed[df_s1_analyzed['area_km2_outlier_combined'] == 1][[
    'time', 'loc', 'image_id', 'sat_id', 'area_km2', 'area_km2_outlier_iqr', 
    'area_km2_outlier_zscore', 'area_km2_outlier_mad', 'area_km2_zscore', 'area_km2_mad_score'
]].sort_values(['loc', 'area_km2'], ascending=[True, False])
outliers_s1.to_csv('s1_grd_area_km2_outliers.csv', index=False)
quality_s1.to_csv('s1_grd_quality_report.csv', index=False)
print(f"S1_GRD: {len(outliers_s1)} outliers exported")

# Landsat area_km2
outliers_landsat_area = df_landsat_analyzed[df_landsat_analyzed['area_km2_outlier_combined'] == 1][[
    'time', 'loc', 'image_id', 'sat_id', 'area_km2', 'area_km2_outlier_iqr', 
    'area_km2_outlier_zscore', 'area_km2_outlier_mad', 'area_km2_zscore', 'area_km2_mad_score'
]].sort_values(['loc', 'area_km2'], ascending=[True, False])
outliers_landsat_area.to_csv('landsat_area_km2_outliers.csv', index=False)
quality_landsat_area.to_csv('landsat_area_km2_quality_report.csv', index=False)
print(f"Landsat area_km2: {len(outliers_landsat_area)} outliers exported")

# Landsat ndwi_area_km2
if 'ndwi_area_km2_outlier_combined' in df_landsat_analyzed.columns:
    outliers_landsat_ndwi = df_landsat_analyzed[df_landsat_analyzed['ndwi_area_km2_outlier_combined'] == 1][[
        'time', 'loc', 'image_id', 'sat_id', 'ndwi_area_km2', 'ndwi_area_km2_outlier_iqr', 
        'ndwi_area_km2_outlier_zscore', 'ndwi_area_km2_outlier_mad', 'ndwi_area_km2_zscore', 'ndwi_area_km2_mad_score'
    ]].sort_values(['loc', 'ndwi_area_km2'], ascending=[True, False])
    outliers_landsat_ndwi.to_csv('landsat_ndwi_area_km2_outliers.csv', index=False)
    quality_landsat_ndwi.to_csv('landsat_ndwi_area_km2_quality_report.csv', index=False)
    print(f"Landsat ndwi_area_km2: {len(outliers_landsat_ndwi)} outliers exported")

print("\nAll files exported successfully!")

EXPORTING RESULTS
S1_GRD: 2588 outliers exported
Landsat area_km2: 2013 outliers exported
Landsat ndwi_area_km2: 416 outliers exported

All files exported successfully!
